In [1]:
import os
from PIL import Image
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
# Set VM data path and read TRAINING labels file
dataPath = '/scratch/data/images/'
df_ind = pd.read_csv('../../../data/processed/test-train/train.csv')
df_ind.head()

,index,filename,label1,label2,label3
0,1,Pia1.2016-10-04.1801+N292_hc.tif,zooplankton,noncopepod,annelida_polychaeta
1,2,Pia1.2016-10-05.1229+N28_hc.tif,zooplankton,noncopepod,annelida_polychaeta
2,3,Pia1.2016-10-06.2118+N136_hc.tif,zooplankton,noncopepod,annelida_polychaeta
3,4,Pia1.2017-03-21.1136+N01644266_hc.tif,zooplankton,noncopepod,annelida_polychaeta
4,5,Pia1.2017-03-21.1136+N01646706_hc.tif,zooplankton,noncopepod,annelida_polychaeta


In [3]:
respath = '/output/resnet50/classes_acc.csv'
df_res = pd.read_csv(respath, header=None)
df_res[df_res[2]<0.7][[1, 2]]

,1,2
3,byrozoa-larvae,0.6667
9,copepod_calanoida,0.4518
10,copepod_calanoida_acartia-spp,0.4667
12,copepod_calanoida_candacia-spp,0.5000
13,copepod_calanoida_centropages-spp,0.6753
15,copepod_calanoida_temora-spp,0.3125
16,copepod_cyclopoida,0.2000
18,copepod_cyclopoida_oithona-spp,0.6327
19,copepod_cyclopoida_oncaea-spp,0.6620
20,copepod_harpacticoida,0.6719


In [4]:
bad_list = df_res[df_res[2]<0.7][1].tolist()
bad_list

['byrozoa-larvae',
 'copepod_calanoida',
 'copepod_calanoida_acartia-spp',
 'copepod_calanoida_candacia-spp',
 'copepod_calanoida_centropages-spp',
 'copepod_calanoida_temora-spp',
 'copepod_cyclopoida',
 'copepod_cyclopoida_oithona-spp',
 'copepod_cyclopoida_oncaea-spp',
 'copepod_harpacticoida',
 'copepod_unknown',
 'decapoda-larvae_brachyura',
 'euphausiid_nauplii']

In [5]:
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

datagen = ImageDataGenerator(
        rotation_range=360,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')

In [10]:
savePath = '/output/data/augment/'

for b in bad_list:
    augPath = savePath + str(b) + '/'
    if not os.path.exists(augPath):
        os.makedirs(augPath)
        im_list = df_ind[df_ind['label3']==b]['filename'].tolist() 
        
        if len(im_list)>=1000:
            mag = 2
        elif len(im_list)<1000 and len(im_list)>=500:
            mag = 3
        elif len(im_list)<500 and len(im_list)>=200:
            mag = 5
        else:
            mag = 10
            
        print(augPath, ":", len(im_list), mag)
        
        for f in im_list:
            im = load_img(dataPath + f)
            im = im.resize((256,256))
            x = img_to_array(im)  
            x = x.reshape((1,) + x.shape)

            # the .flow() command below generates batches of randomly transformed images
            # and saves the results to directory
            i = 0
            for batch in datagen.flow(x, batch_size=1,
                                      save_to_dir=augPath, save_prefix=f+"_aug", save_format='png'):
                i += 1
                if i >= mag:
                    break  # otherwise the generator would loop indefinitely
        print("after aug:", len(os.listdir(augPath)))
    else:
        print(augPath, ":", len(os.listdir(augPath)))

/output/data/augment/byrozoa-larvae/ : 870
/output/data/augment/copepod_calanoida/ : 2982
/output/data/augment/copepod_calanoida_acartia-spp/ : 405 5
after aug: 2024
/output/data/augment/copepod_calanoida_candacia-spp/ : 36 10
after aug: 360
/output/data/augment/copepod_calanoida_centropages-spp/ : 692 3
after aug: 2076
/output/data/augment/copepod_calanoida_temora-spp/ : 147 10
after aug: 1469
/output/data/augment/copepod_cyclopoida/ : 88 10
after aug: 880
/output/data/augment/copepod_cyclopoida_oithona-spp/ : 441 5
after aug: 2205
/output/data/augment/copepod_cyclopoida_oncaea-spp/ : 630 3
after aug: 1890
/output/data/augment/copepod_harpacticoida/ : 575 3
after aug: 1725
/output/data/augment/copepod_unknown/ : 1648 2
after aug: 3296
/output/data/augment/decapoda-larvae_brachyura/ : 36 10
after aug: 359
/output/data/augment/euphausiid_nauplii/ : 126 10
after aug: 1260


In [19]:
savePath = '/output/data/augment/'

name_list, before_list, after_list = [], [], []

for b in bad_list:
    augPath = savePath + str(b) + '/'
    im_list = df_ind[df_ind['label3']==b]['filename'].tolist() 
    
    name_list.append(b)
    before_list.append(len(im_list))
    after_list.append(len(os.listdir(augPath)))

    print(b, "&", len(im_list), "&", len(os.listdir(augPath)))

byrozoa-larvae & 87 & 957
copepod_calanoida & 1491 & 4481
copepod_calanoida_acartia-spp & 405 & 2430
copepod_calanoida_candacia-spp & 36 & 397
copepod_calanoida_centropages-spp & 692 & 2772
copepod_calanoida_temora-spp & 147 & 1621
copepod_cyclopoida & 88 & 970
copepod_cyclopoida_oithona-spp & 441 & 2648
copepod_cyclopoida_oncaea-spp & 630 & 2529
copepod_harpacticoida & 575 & 2304
copepod_unknown & 1648 & 4964
decapoda-larvae_brachyura & 36 & 395
euphausiid_nauplii & 126 & 1386


In [20]:
df_aug = pd.DataFrame([name_list, before_list, after_list]).T
df_aug.columns = ['label3', 'size before augmentation', 'size after augmentation']
df_aug

,label3,size before augmentation,size after augmentation
0,byrozoa-larvae,87,957
1,copepod_calanoida,1491,4481
2,copepod_calanoida_acartia-spp,405,2430
3,copepod_calanoida_candacia-spp,36,397
4,copepod_calanoida_centropages-spp,692,2772
5,copepod_calanoida_temora-spp,147,1621
6,copepod_cyclopoida,88,970
7,copepod_cyclopoida_oithona-spp,441,2648
8,copepod_cyclopoida_oncaea-spp,630,2529
9,copepod_harpacticoida,575,2304
